In [14]:
from tabula import read_pdf
import pandas as pd
import re

df_temp = read_pdf(r"C:\Users\mkfal\Documents\GitHub\BI\MP1\dat4bi-mp1\data\usage-history.pdf", stream=True, pages="all")

def mask_phone_like(text: str, keep_last: int = 3, min_len: int = 7) -> str:
    """Mask all digits except the last keep_last if total digits >= min_len, preserving separators."""
    s = str(text)
    total_digits = sum(ch.isdigit() for ch in s)
    if total_digits < min_len:
        return s
    to_mask = total_digits - keep_last
    out = []
    for ch in s:
        if ch.isdigit() and to_mask > 0:
            out.append("X")
            to_mask -= 1
        else:
            out.append(ch)
    return "".join(out)

def mask_modtager_or_phone_cols(df: pd.DataFrame) -> pd.DataFrame:
    cols = [c for c in df.columns if c is not None]
    norm = {c: re.sub(r"\s+", " ", str(c)).strip().lower() for c in cols}

    # Prefer columns named or containing "modtager"
    modtager_cols = [c for c in cols if "modtager" in norm[c]]

    # Never mask date/time-like columns by name
    EXCLUDE_BY_NAME = {"dato", "date", "tid", "time", "datetime"}

    def is_date_like(val: str) -> bool:
        s = str(val).strip()
        if not s:
            return False
        # dd-mm-yyyy, dd/mm/yyyy, dd.mm.yyyy, yyyy-mm-dd, with optional time
        if re.match(r"^\d{1,2}[./-]\d{1,2}[./-]\d{2,4}(\s+\d{1,2}:\d{2}(:\d{2})?)?$", s):
            return True
        if re.match(r"^\d{4}[./-]\d{1,2}[./-]\d{1,2}(\s+\d{1,2}:\d{2}(:\d{2})?)?$", s):
            return True
        return False

    def is_phone_like(val: str, min_len: int = 7, max_len: int = 15) -> bool:
        s = str(val)
        if is_date_like(s):
            return False
        digits = sum(ch.isdigit() for ch in s)
        return min_len <= digits <= max_len

    if not modtager_cols:
        candidate_cols = []
        for c in cols:
            if any(tok in norm[c] for tok in EXCLUDE_BY_NAME):
                continue
            s = df[c].dropna().astype(str)
            if len(s) == 0:
                continue
            ratio_phone = (s.map(is_phone_like)).mean()
            ratio_date = (s.map(is_date_like)).mean()
            if ratio_phone >= 0.30 and ratio_date < 0.50:
                candidate_cols.append(c)
        modtager_cols = candidate_cols

    for c in modtager_cols:
        df[c] = df[c].astype(str).map(mask_phone_like)
    return df

# Apply masking whether read_pdf returned a DataFrame or a list of DataFrames
if isinstance(df_temp, list):
    df_temp = [mask_modtager_or_phone_cols(d) for d in df_temp if isinstance(d, pd.DataFrame)]
    # Optional: concatenate for easier viewing
    try:
        df_view = pd.concat(df_temp, ignore_index=True)
    except Exception:
        df_view = df_temp
else:
    df_temp = mask_modtager_or_phone_cols(df_temp)
    df_view = df_temp

# Debug: show detected columns and sample
if isinstance(df_view, pd.DataFrame):
    print("Columns:", list(df_view.columns))
    print(df_view.head(10).to_string(index=False))
else:
    print(df_view)

Columns: ['Opkald', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Dato', 'Tidspunkt', 'Varighed', 'Modtager', 'Pris', 'Data forbrug']
    Opkald Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Dato Tidspunkt Varighed Modtager Pris Data forbrug
      Dato  Tidspunkt   Varighed   Modtager       Pris  NaN       NaN      NaN      NaN  NaN          NaN
22/07/2025      15:56   00:00:16 XXXXXXX573          0  NaN       NaN      NaN      NaN  NaN          NaN
15/07/2025      16:53   00:00:23 XXXXXXX573          0  NaN       NaN      NaN      NaN  NaN          NaN
15/07/2025      11:07   00:01:08 XXXXXXX573          0  NaN       NaN      NaN      NaN  NaN          NaN
11/07/2025      12:55   00:02:56 XXXXXXX470          0  NaN       NaN      NaN      NaN  NaN          NaN
11/07/2025      12:54   00:00:04 XXXXXXX573          0  NaN       NaN      NaN      NaN  NaN          NaN
11/07/2025      12:53   00:00:04 XXXXXXX573          0  NaN       NaN      NaN      NaN  NaN          NaN
09/07/2